#Mounting

In [0]:
storageAccountName = ""
storageAccountAccessKey = ''
sasToken = ''
blobContainerName = "rag-files"
mountPoint = "/mnt/data/"
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
  try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
      mount_point = mountPoint,
      extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
    )
    print("mount succeeded!")
  except Exception as e:
    print("mount exception", e)

mount succeeded!


In [0]:
dbutils.fs.mounts()

[MountInfo(mountPoint='/databricks-datasets', source='databricks-datasets', encryptionType=''),
 MountInfo(mountPoint='/Volumes', source='UnityCatalogVolumes', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-tracking', source='databricks/mlflow-tracking', encryptionType=''),
 MountInfo(mountPoint='/databricks-results', source='databricks-results', encryptionType=''),
 MountInfo(mountPoint='/databricks/mlflow-registry', source='databricks/mlflow-registry', encryptionType=''),
 MountInfo(mountPoint='/Volume', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/volumes', source='DbfsReserved', encryptionType=''),
 MountInfo(mountPoint='/mnt/data/', source='wasbs://rag-files@rahulsadls.blob.core.windows.net', encryptionType=''),
 MountInfo(mountPoint='/', source='DatabricksRoot', encryptionType=''),
 MountInfo(mountPoint='/volume', source='DbfsReserved', encryptionType='')]

In [0]:
!pip install transformers pypdf sentence-transformers

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
spark.conf.set(
    "fs.azure.account.key.<storage_account>.blob.core.windows.net",
    "<storage_account_access_token>"
)

In [0]:
dbutils.fs.ls("/mnt/data/Machine learning - Wikipedia.pdf")

[FileInfo(path='dbfs:/mnt/data/Machine learning - Wikipedia.pdf', name='Machine learning - Wikipedia.pdf', size=2318840, modificationTime=1733143863000)]

#Extract Text from PDF

In [0]:
from pypdf import PdfReader
reader = PdfReader("Machine learning - Wikipedia.pdf")
text = ""
for page in reader.pages:
    text += page.extract_text()
print(text)

Machine learning
Machine learning (ML) is a field of study in artificial intelligence concerned with the development
and study of statistical algorithms that can learn from data and generalize to unseen data, and thus
perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed
neural networks to surpass many previous approaches in performance.[2]
ML finds application in many fields, including natural language processing, computer vision, speech
recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business
problems is known as predictive analytics.
Statistics and mathematical optimization (mathematical programming) methods comprise the
foundations of machine learning. Data mining is a related field of study, focusing on exploratory data
analysis (EDA) via unsupervised learning.[6][7]
From a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework
for describing machine learning.
T

In [0]:
from transformers import AutoTokenizer, AutoModel

2024-12-02 13:14:47.939747: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 13:14:48.253102: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Generating Embeddings

In [0]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
print("Tokenized Sample:", inputs)

Tokenized Sample: {'input_ids': tensor([[37573,  4673,   198, 37573,  4673,   357,  5805,     8,   318,   257,
          2214,   286,  2050,   287, 11666,  4430,  5213,   351,   262,  2478,
           198,   392,  2050,   286, 13905, 16113,   326,   460,  2193,   422,
          1366,   290,  2276,  1096,   284, 29587,  1366,    11,   290,  4145,
           198,   525,   687,  8861,  1231,  7952,  7729,  3693,    16,    60,
          8007,  1817,   287,   262,  2214,   286,  2769,  4673,   423,  3142,
           198,   710,  1523,  7686,   284, 17341,   867,  2180, 10581,   287,
          2854,  3693,    17,    60,   198,  5805,  7228,  3586,   287,   867,
          7032,    11,  1390,  3288,  3303,  7587,    11,  3644,  5761,    11,
          4046,   198, 26243,   653,    11,  3053, 25431,    11, 14510,    11,
           290,  9007,  3693,    18,  7131,    19,    60,   383,  3586,   286,
         10373,   284,  1597,   198,  1676, 22143,   318,  1900,   355, 33344,
         23696,    1